In [57]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
def sep_two_classes(Y, num_class):
    Y = Y.copy()
    for i in range(Y.shape[0]):
        if Y[i] == num_class:
            Y[i] = 1
        else:
            Y[i] = -1
    return Y

In [4]:
def h(theta, X):
    return 1.0 / (1 + np.exp(-np.dot(X, theta)))

In [5]:
def weak_clssefier(X, weights):
    Z = h(weights, X)
    return Z

In [54]:
def train_clf(X, Y, weights):
    newY = np.asarray([y if y == 1 else 0 for y in Y])
    P = np.asarray(h(weights, X))
    L = 0
    _accuracy = 0
    alpha = 0.001
    t = 0
    while t < 10:
        for i in range(len(weights)):
            weights[i] -= alpha * sum([(P[j] - newY[j])*X[j][i] for j in range(len(P))])
        P = h(weights, X)
        _accuracy = accuracy(newY, [0 if p < 0.5 else 1 for p in P])
        alpha = max(0.0001, alpha*0.9)
        if _accuracy > 0.5:
            return weights
        t+=1
    return weights

In [7]:
def Q(X, Y, W, weights_for_weak_clf):
    Qsum = 0
    for i in range(X.shape[0]):
        p = -1 if weak_clssefier(X[i], weights_for_weak_clf) < 0.5 else 1
        I = 1 if (Y[i]*p) < 0 else 0
        Qsum += W[i] * I
    return Qsum

In [8]:
def alpha(Q):
    return 0.5 * np.log((1-Q)/Q)

In [89]:
def fit(X, Y, num_weak_clf, T):
    #1 инициализация весов объектов
    num1 = 0
    for l in Y:
        if l == 1:
            num1 +=1
    
    P = np.asarray([1/num1 if y == 1 else 1/(Y.shape[0] - num1) for y in range(Y.shape[0])])
    alphas = []
    bs = []
    #2 Для всех t=1,...,T, пока не выполнен критерий останова.
    while len(alphas) < num_weak_clf:
        weights_for_weak_cls = np.random.rand(T, X.shape[1])#инициализируем матрицу с весами для слабых классификаторов,
        print(weights_for_weak_cls)
                        #выберем тот, у которого самые хорошие веса
        #print('WFCLS', weights_for_weak_cls)
        #2.1 Находим классификатор b_t: X->{-1,+1} который минимизирует взвешенную ошибку классификации;
        #b_t = argmin_b Q(b,W^l);
        for wfwc in weights_for_weak_cls:
            wfwc = train_clf(X, Y, wfwc)
        Qs = []
        for wfwc in weights_for_weak_cls:
            Qs.append(Q(X, Y, P, wfwc))
        Qs = np.asarray(Qs)
        argmin = np.argmin(Qs)
        if Qs[argmin]> 0.5:
            pass
        else:
            b_t_weights = weights_for_weak_cls[argmin]
            bs.append(b_t_weights)
            #2.2 Пересчитываем кооэффициент взвешенного голосования для алгоритма классификации b_t:

            alpha_t = alpha(Qs[argmin])
            alphas.append(alpha_t)

            #2.3 Пересчет весов объектов: w_i = w_i*exp(-alpha_t*y_i*b_t(x_i)), i = 1,...,l
            for i in range(P.shape[0]):
                p = -1 if weak_clssefier(X[i], b_t_weights) < 0.5 else 1
                P[i] *= np.exp(-alpha_t*Y[i]*p)

            #2.4 Нормировка весов объектов:

            P0 = np.sum(P)
            for p in P:
                p/=P0
    return np.asarray(alphas), np.asarray(bs)
        

In [10]:
def predict(X, alphas, weak_clf_weights):
    tmp_sum = 0
    for t in range(alphas.shape[0]):
        p = -1 if weak_clssefier(X, weak_clf_weights[t]) < 0.5 else 1
        tmp_sum += alphas[t] * p
    return (np.sign(tmp_sum), tmp_sum)

In [11]:
def accuracy(y_true, y_pred):
    counter = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            counter +=1
    return counter/len(y_true)

In [78]:
def plot_progress(X, Y, alphas, bs):
    progress = [0,]
    for i in range(0, len(alphas[0])):
        ttl = []
        for x in X:
            pr0 = predict(x, alphas[0][0:i], bs[0][0:i, :])
            pr1 = predict(x, alphas[1][0:i], bs[1][0:i, :])
            pr2 = predict(x, alphas[2][0:i], bs[2][0:i, :])
            ttl.append(np.argmax(np.asarray([pr0[1], pr1[1], pr2[1]])))
        progress.append(accuracy(Y, ttl))
    plt.plot(progress)
    plt.yticks(np.arange(0, 1.0, 0.1))
    plt.show()

In [13]:
iris = datasets.load_iris()
X, Y = iris.data, iris.target
p=np.random.permutation(X.shape[0])
x_train = X[p[0:int(len(X)*0.7)], :]
y_train = Y[p[0:int(len(Y)*0.7)]]
x_test = X[p[int(len(X)*0.7):], :]
y_test = Y[p[int(len(Y)*0.7):]]
x_train = (x_train - np.mean(x_train, axis = 0)) / np.var(x_train, axis = 0)
x_test = (x_test - np.mean(x_test, axis = 0)) / np.var(x_test, axis = 0)

y_train0 = sep_two_classes(y_train, 0)
y_train1 = sep_two_classes(y_train, 1)
y_train2 = sep_two_classes(y_train, 2)

y_test0 = sep_two_classes(y_test, 0)
y_test1 = sep_two_classes(y_test, 1)
y_test2 = sep_two_classes(y_test, 2)

In [ ]:
accuracys0 = np.asarray([])
accuracys1 = np.asarray([])
accuracys2 = np.asarray([])
total_accs = np.asarray([])
np.random.seed(0)
num_wclfs = 5
print('\n\n\n----------------------------------------------')
start_time = time.time()
alphas0, bs0 = fit(x_train, y_train0, num_wclfs, 15)
alphas1, bs1 = fit(x_train, y_train1, num_wclfs, 15)
alphas2, bs2 = fit(x_train, y_train2, num_wclfs, 15)
print("--- %s seconds ---" % (time.time() - start_time))

plot_progress(x_test, y_test, [alphas0, alphas1, alphas2], [bs0, bs1, bs2])
y_predicted0 = []
y_predicted1 = []
y_predicted2 = []
total_pr = []
for x in x_test:
    pr0 = predict(x, alphas0, bs0)
    pr1 = predict(x, alphas1, bs1)
    pr2 = predict(x, alphas2, bs2)
    y_predicted0.append(pr0[0])
    y_predicted1.append(pr1[0])
    y_predicted2.append(pr2[0])
    total_pr.append(np.argmax(np.asarray([pr0[1], pr1[1], pr2[1]])))

y_predicted0 = np.asarray(y_predicted0)
y_predicted1 = np.asarray(y_predicted1)
y_predicted2 = np.asarray(y_predicted2)
total_pr = np.asarray(total_pr)

acc0 = accuracy(y_test0, y_predicted0)
acc1 = accuracy(y_test1, y_predicted1)
acc2 = accuracy(y_test2, y_predicted2)
total_acc = accuracy(y_test, total_pr)

np.append(accuracys0, acc0)
np.append(accuracys1, acc1)
np.append(accuracys2, acc2)
np.append(total_accs, total_acc)
print('accurecy: {T} weak clfs'.format(T=num_wclfs), '\n 0:|{acc0}| \n 1:|{acc1}| \n 2:|{acc2}| \n total:|{total_acc}| \n'.format(acc0=acc0, acc1=acc1, acc2=acc2, total_acc=total_acc))




----------------------------------------------
[[0.5488135  0.71518937 0.60276338 0.54488318]
 [0.4236548  0.64589411 0.43758721 0.891773  ]
 [0.96366276 0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606 0.0871293 ]
 [0.0202184  0.83261985 0.77815675 0.87001215]
 [0.97861834 0.79915856 0.46147936 0.78052918]
 [0.11827443 0.63992102 0.14335329 0.94466892]
 [0.52184832 0.41466194 0.26455561 0.77423369]
 [0.45615033 0.56843395 0.0187898  0.6176355 ]
 [0.61209572 0.616934   0.94374808 0.6818203 ]
 [0.3595079  0.43703195 0.6976312  0.06022547]
 [0.66676672 0.67063787 0.21038256 0.1289263 ]
 [0.31542835 0.36371077 0.57019677 0.43860151]
 [0.98837384 0.10204481 0.20887676 0.16130952]
 [0.65310833 0.2532916  0.46631077 0.24442559]]
[[0.15896958 0.11037514 0.65632959 0.13818295]
 [0.19658236 0.36872517 0.82099323 0.09710128]
 [0.83794491 0.09609841 0.97645947 0.4686512 ]
 [0.97676109 0.60484552 0.73926358 0.03918779]
 [0.28280696 0.12019656 0.2961402  0.11872772]
 [0.31798

[[2.35834224e-01 6.20499900e-01 6.39622243e-01 9.48540301e-01]
 [7.78276167e-01 8.48345270e-01 4.90419908e-01 1.85348587e-01]
 [9.95815293e-01 1.29355761e-01 4.71457319e-01 6.80930992e-02]
 [9.43850857e-01 9.64924941e-01 7.19389062e-01 3.49992844e-01]
 [2.54382401e-01 2.65303325e-01 1.27294025e-01 5.25808953e-01]
 [1.41817276e-01 3.16730667e-01 6.26706476e-01 7.27543610e-01]
 [2.42727046e-02 4.30115984e-01 6.52124595e-01 8.53245976e-01]
 [4.75324782e-01 9.69205872e-01 2.65632548e-01 1.35087066e-02]
 [4.83752865e-01 2.56113795e-01 8.23717672e-01 2.32772672e-01]
 [3.10629218e-01 7.91227431e-01 7.15143252e-01 5.58051237e-01]
 [7.04948062e-01 4.18636864e-01 5.31004761e-03 1.13551285e-02]
 [5.11221788e-01 8.32909797e-02 5.10754802e-02 9.65516639e-01]
 [8.59002640e-01 1.52027227e-01 6.64218590e-04 9.41667795e-01]
 [2.78325298e-01 1.85897603e-01 6.91508108e-01 1.08903739e-01]
 [2.64649598e-01 9.75094680e-01 6.39462774e-01 5.20677791e-01]]
[[0.39791861 0.77450095 0.14095748 0.9673378 ]
 [0.861

[[6.01943698e-01 6.33690045e-02 4.09857450e-01 7.22500088e-01]
 [2.38738841e-01 9.43827589e-01 6.86783368e-01 2.87575383e-01]
 [7.68998923e-01 8.31647718e-02 9.74774423e-01 4.92852591e-02]
 [9.33455891e-01 2.52853878e-01 7.57824108e-01 7.36994331e-05]
 [2.54240089e-01 7.49100607e-01 5.32336071e-01 1.14952150e-01]
 [3.93629746e-01 3.75549355e-01 5.68162244e-01 6.67977072e-01]
 [8.40830242e-01 4.97231397e-01 3.92021717e-01 1.43976534e-01]
 [8.04822965e-01 7.13370405e-01 4.08677397e-01 5.18432310e-01]
 [6.65182844e-01 1.64805590e-01 2.71977944e-02 3.17503699e-01]
 [5.95585020e-01 4.86606091e-01 6.92554627e-01 8.19689806e-01]
 [4.88442467e-01 1.34267024e-01 8.50628000e-01 5.74990327e-01]
 [7.39937481e-01 7.04664654e-01 9.68211771e-01 2.95307322e-01]
 [7.05306773e-01 3.65676333e-01 3.95410721e-01 2.30594637e-01]
 [3.44010176e-01 9.48296752e-01 2.92570847e-01 2.45990606e-01]
 [5.83137978e-01 2.58035957e-01 4.73385726e-01 8.34176257e-01]]
[[0.23040031 0.42669141 0.61048974 0.54562892]
 [0.974

[[0.4139625  0.6296183  0.77858426 0.85155779]
 [0.81641271 0.16607709 0.82838955 0.05862906]
 [0.20017068 0.62292672 0.11469253 0.6033476 ]
 [0.30796569 0.42946659 0.3151515  0.07017357]
 [0.50075875 0.96359944 0.61178335 0.30003187]
 [0.95614983 0.38923725 0.69751773 0.6341128 ]
 [0.95291085 0.91460269 0.14800406 0.08568587]
 [0.80964233 0.90494321 0.4929736  0.69076127]
 [0.06978828 0.2198044  0.14937619 0.60116058]
 [0.19753989 0.05984632 0.30220728 0.67721243]
 [0.57786372 0.33072497 0.83347888 0.27626911]
 [0.26054764 0.55994741 0.47821564 0.44597266]
 [0.15578562 0.50464619 0.25901892 0.39217988]
 [0.73357045 0.93686283 0.77386297 0.40563389]
 [0.59303314 0.74649782 0.41422105 0.36024816]]
[[0.1352433  0.7362608  0.83850859 0.31658441]
 [0.78350777 0.61021306 0.78408305 0.41133738]
 [0.74757636 0.40850912 0.08875799 0.55057892]
 [0.45432873 0.25287989 0.89211333 0.38285331]
 [0.03912611 0.38213574 0.42450981 0.86163772]
 [0.79918321 0.19996659 0.83902923 0.24792723]
 [0.34193415

[[3.96905539e-01 1.82951940e-01 8.77940510e-01 8.24077224e-01]
 [7.74959895e-01 5.72900385e-02 5.56719442e-01 2.35013562e-01]
 [1.19685197e-01 7.81613100e-01 2.11581375e-01 5.00811698e-01]
 [5.26096348e-01 2.13282004e-01 4.92261166e-01 5.28198976e-02]
 [7.24496385e-05 8.80321932e-01 4.15588702e-01 9.07419799e-01]
 [4.96307360e-01 4.31108295e-01 2.95912141e-01 8.16889376e-02]
 [3.09694649e-01 6.08470265e-01 7.62786347e-01 5.40825518e-01]
 [7.58255844e-01 1.57751592e-01 5.83239932e-01 8.82603227e-01]
 [2.17960208e-01 2.27668938e-01 7.48432513e-01 8.12056785e-01]
 [2.48046330e-01 3.31569804e-01 6.31769400e-01 3.98431713e-01]
 [6.24010970e-01 4.96430032e-01 7.73651309e-01 8.45888647e-01]
 [8.63827209e-01 6.11525078e-01 6.12584105e-01 4.49646899e-01]
 [2.76140450e-02 2.99180219e-01 6.36977185e-02 4.37785249e-01]
 [8.49485249e-01 5.96802018e-01 6.75532598e-01 1.88007972e-01]
 [2.61953150e-01 1.63132153e-01 8.42688762e-01 2.80313013e-01]]
[[0.56232631 0.14112861 0.12398891 0.34758336]
 [0.330

...

[[0.21934811 0.1568173  0.54893311 0.37770939]
 [0.50730431 0.82047857 0.79330136 0.66053329]
 [0.13619176 0.03733315 0.3353388  0.09320918]
 [0.41540729 0.28058941 0.79033478 0.83922054]
 [0.97795529 0.97070121 0.0215116  0.4107067 ]
 [0.99848633 0.35247573 0.46309437 0.77482014]
 [0.73709644 0.83360917 0.26517631 0.02058388]
 [0.62605171 0.69042777 0.53758529 0.13897403]
 [0.28051359 0.11255692 0.34647093 0.18569383]
 [0.49093526 0.35536351 0.15739719 0.35767802]
 [0.8827913  0.26235998 0.39409041 0.17280813]
 [0.79662955 0.62111751 0.00391825 0.44887951]
 [0.97699692 0.91813031 0.95844318 0.92897596]
 [0.62792283 0.32810998 0.78087493 0.00423733]
 [0.45571552 0.89285165 0.2696475  0.0757085 ]]
[[0.10365102 0.42505379 0.76269917 0.48757048]
 [0.50061482 0.56261929 0.36707506 0.31860036]
 [0.10819826 0.67417629 0.20233629 0.28878553]
 [0.94715751 0.57208434 0.84691027 0.44759465]
 [0.63263025 0.36998147 0.81103724 0.03438122]
 [0.7854339  0.83718575 0.82337944 0.82893334]
 [0.91430825

[[0.86168212 0.95220313 0.28826677 0.01078892]
 [0.49405625 0.06824899 0.26582599 0.79347444]
 [0.7298492  0.72064577 0.02691646 0.45965571]
 [0.17641815 0.87786515 0.79844014 0.08010308]
 [0.69275427 0.57324423 0.26654329 0.80433892]
 [0.95443889 0.04367019 0.47834765 0.41158014]
 [0.9631415  0.53533483 0.4198839  0.0622449 ]
 [0.28448244 0.06760904 0.60338899 0.74820808]
 [0.57233077 0.91813213 0.59242066 0.42787837]
 [0.28945925 0.3290741  0.23493878 0.76269542]
 [0.17784266 0.17237426 0.08132322 0.31606624]
 [0.35079145 0.58158936 0.74489179 0.57421146]
 [0.57486369 0.89571275 0.36725307 0.94167152]
 [0.16568101 0.03966088 0.67490808 0.15039694]
 [0.69795432 0.57022225 0.21557965 0.39246478]]
[[0.5064459  0.4052265  0.93919457 0.87952949]
 [0.6800866  0.27583937 0.00784385 0.97888766]
 [0.98405621 0.32841131 0.88178677 0.54433982]
 [0.82081107 0.93950091 0.3248486  0.80652905]
 [0.71813285 0.65835961 0.16086113 0.99065218]
 [0.31869888 0.85390904 0.57225227 0.16793702]
 [0.40383177

[[0.96503278 0.07731355 0.13053471 0.35373281]
 [0.51898368 0.60877635 0.04064148 0.7269944 ]
 [0.61052641 0.6753018  0.54490664 0.90717212]
 [0.72723792 0.73180491 0.60034909 0.4358914 ]
 [0.98142117 0.30618792 0.95349441 0.72959463]
 [0.05415631 0.72773259 0.38895717 0.74943906]
 [0.23931438 0.95543374 0.67374632 0.52626962]
 [0.45011909 0.89351003 0.64266411 0.81873832]
 [0.8020514  0.40589052 0.74007197 0.16939835]
 [0.28212368 0.73311485 0.8623966  0.28201724]
 [0.72409474 0.85401172 0.94794604 0.33957777]
 [0.14702213 0.8523242  0.1860048  0.14666208]
 [0.92990309 0.69062583 0.63196769 0.76941512]
 [0.53413683 0.78058674 0.69132619 0.61094987]
 [0.22166178 0.12055847 0.94735328 0.49929837]]
[[0.43745877 0.48204265 0.05140024 0.06353678]
 [0.36274779 0.95790621 0.04000977 0.45665526]
 [0.33635909 0.39191429 0.25895594 0.22777972]
 [0.13971417 0.28426157 0.85832439 0.61460525]
 [0.82363347 0.78007211 0.4679997  0.71052545]
 [0.8275096  0.38180443 0.38827178 0.49939258]
 [0.09067702

[[0.88534527 0.28327923 0.68092334 0.97206235]
 [0.07439443 0.11488542 0.5743016  0.91290479]
 [0.32044026 0.85154823 0.1003847  0.61442124]
 [0.90825552 0.35382754 0.19266124 0.97769456]
 [0.00634992 0.78226832 0.07327668 0.81043628]
 [0.49844407 0.82904498 0.03343076 0.75386687]
 [0.44327018 0.20436455 0.18899482 0.79064474]
 [0.66701111 0.657605   0.52913291 0.54029299]
 [0.23828647 0.94591123 0.04265601 0.687844  ]
 [0.01791824 0.62201349 0.27941231 0.98579146]
 [0.15073485 0.72829149 0.42748267 0.22477931]
 [0.8942363  0.09729541 0.24156245 0.89906115]
 [0.7021793  0.96523478 0.09045659 0.70597611]
 [0.06393988 0.34953925 0.51087664 0.84554625]
 [0.77597574 0.95344649 0.20039203 0.56543225]]
[[0.28794821 0.87828266 0.56667037 0.15192268]
 [0.15104076 0.97519395 0.12917543 0.23832445]
 [0.25984507 0.90465872 0.57290764 0.23049795]
 [0.98531749 0.20801795 0.17216151 0.91331352]
 [0.39326592 0.7504195  0.93971172 0.47074486]
 [0.97495476 0.87590531 0.3302584  0.33622724]
 [0.84208138

[[0.54059898 0.99110811 0.66111562 0.6584851 ]
 [0.8796598  0.36721608 0.46513722 0.26690836]
 [0.19815784 0.53533053 0.66605848 0.56172478]
 [0.35234021 0.84109384 0.15034692 0.11270335]
 [0.61896164 0.07100092 0.96777203 0.91978209]
 [0.36221066 0.84729439 0.08034477 0.07660656]
 [0.54918601 0.55425359 0.4060225  0.78414953]
 [0.10891744 0.90953483 0.02928455 0.80128354]
 [0.55678699 0.62725822 0.10894238 0.23039582]
 [0.7346246  0.64368542 0.81245765 0.52684894]
 [0.20759543 0.0991128  0.54323813 0.32170049]
 [0.63929619 0.31520324 0.4525042  0.62890032]
 [0.18903002 0.51862684 0.0353764  0.19511456]
 [0.40866221 0.65000905 0.55738383 0.17421744]
 [0.8380166  0.79877416 0.34943515 0.12292109]]
[[0.27555452 0.82385689 0.80599122 0.96386981]
 [0.14686967 0.22209008 0.47052878 0.17830485]
 [0.49804612 0.78227009 0.25424495 0.07370566]
 [0.3614242  0.19039513 0.90152444 0.40913824]
 [0.47750197 0.50040787 0.28505338 0.96223176]
 [0.67428931 0.81447473 0.25660946 0.38076908]
 [0.44312822